In [2]:
%load_ext autoreload
%autoreload 2

import torch
import wandb
import matplotlib.pyplot as plt

from text3d2video.animation_artifact import AnimationArtifact
from latentman.pipelines.pipeline_latentman import StableDiffusionLatentMan
from latentman.pipelines.scheduling_ddim import DDIMScheduler
from latentman.misc.cross_frame_attn import CrossFrameAttnProcessor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
api = wandb.Api()
animation = api.artifact('romeu/diffusion-3D-features/backflip:latest')
animation = AnimationArtifact.from_wandb_artifact(animation)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Downloading large artifact backflip:latest, 121.09MB. 61 files... 
wandb:   61 of 61 files downloaded.  
Done. 0:0:0.5


In [4]:
from text3d2video.rendering import make_rasterizer
from pytorch3d.renderer import look_at_view_transform, FoVPerspectiveCameras

device = torch.device("cuda:0")

# load camera
R, T = look_at_view_transform(dist=2, azim=0, elev=0)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T, fov=60)

# render meshes
rasterizer = make_rasterizer(cameras)
meshes = animation.load_frames_batched()
fragments = rasterizer(meshes)

/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [92]:
from pytorch3d.io import load_obj, load_objs_as_meshes
from pytorch3d.renderer import TexturesUV
from pytorch3d.structures import Meshes

verts, faces, aux = load_obj("data/mixamo-human.obj")
vert_uvs = aux.verts_uvs
faces_uvs = faces.textures_idx.unsqueeze(0)
empty_texture = torch.zeros(1,2,2,3)

textures = TexturesUV(maps=empty_texture, faces_uvs=faces_uvs, verts_uvs=[vert_uvs])

mesh = Meshes(verts=[verts], faces=[faces.verts_idx], textures=textures)

/home/jorge/miniconda3/envs/thesis/lib/python3.8/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


AttributeError: 'Meshes' object has no attribute 'shape'